In [2]:
import pymssql
import numpy as np
import pandas as pd
import pandas_datareader.data as pdr
import matplotlib.pyplot as plt
def server_connection(tank_seq):
    conn = pymssql.connect(server='sql16ssd-014.localnet.kr', user='i2on11_admin', password='root0826', database='i2on11_admin')
    cursor = conn.cursor()

    conn_sql = 'SELECT signal_time, tank_remain_volume FROM gas_tank_volume_history where tank_seq = ' + tank_seq + 'ORDER BY signal_time DESC'

    # 쿼리 실행
    cursor.execute(conn_sql)

    # 결과 가져오기
    rows = cursor.fetchall()

    # 연결 닫기
    conn.close()
    return rows

In [10]:
def datas_to_diff_cycle_data(datas, tank_seq):
    d_h = pd.DataFrame(datas, columns=['datetime', 'history'])
    d_h.set_index('datetime', inplace=True)
    diff_d_h = d_h.copy()
    diff_d_h = diff_d_h['history'].diff()
    diff_d_h = diff_d_h.dropna()
    diff_d_h[diff_d_h >= 15] = 0
    diff_d_h_weekly = -diff_d_h.resample('W').sum()
    diff_d_h_weekly = diff_d_h_weekly.dropna()
    diff_d_h_weekly = diff_d_h_weekly.drop(diff_d_h_weekly.index[-1])
    del diff_d_h_weekly[diff_d_h_weekly.index.min()]
    del diff_d_h_weekly[diff_d_h_weekly.index.max()]
    max_cycle_value = 150
    diff_cycle = round(50*7/(diff_d_h_weekly),2)
    # print(diff_cycle)
    diff_cycle[np.isneginf(diff_cycle)] = max_cycle_value
    diff_cycle[diff_cycle < 0] = max_cycle_value
    return diff_cycle

In [11]:
import calendar

def get_days_in_month(year, month):
    return calendar.monthrange(year, month)[1]

def day_in_month_set(day_start_year,day_start_month, day_end_year, day_end_month):
    days_in_month = []
    if day_start_year > day_start_year or (day_start_year == day_end_year and day_start_month > day_end_month):
        print("옳지 않는 범위 설정")
        return days_in_month
    if day_start_year == day_end_year:
        days_in_month = [get_days_in_month(day_start_year, month) for month in range(day_start_month, day_end_month+1)]
    else:
        days_in_month = [get_days_in_month(day_start_year, month) for month in range(day_start_month, 13)]
        day_start_year = day_start_year+1
        while day_start_year < day_end_year:
            for month in range(1, 13):
                days_in_month.append(get_days_in_month(day_start_year, month))
                day_start_year = day_start_year+1
        for month in range(1, day_end_month+1):
                days_in_month.append(get_days_in_month(day_start_year, month))
    return days_in_month

In [12]:
def date_to_fraction(date):
    result = []
    for d in date:
        year = d.year
        month = d.month
        day = d.day

        days_in_month = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31]

        if month < 1 or month > 12:
            raise ValueError("Invalid month value")

        days_to_month = days_in_month[month-1]
        day_of_month = sum(days_in_month[:month - 1]) + day
        #result.append(day_of_month)
        result.append(month + ((day-1)/days_to_month))
    return result

In [13]:
import calendar

def get_days_in_month(year, month):
    return calendar.monthrange(year, month)[1]

def day_in_month_set(day_start_year,day_start_month, day_end_year, day_end_month):
    days_in_month = []
    if day_start_year > day_start_year or (day_start_year == day_end_year and day_start_month > day_end_month):
        print("옳지 않는 범위 설정")
        return days_in_month
    if day_start_year == day_end_year:
        days_in_month = [get_days_in_month(day_start_year, month) for month in range(day_start_month, day_end_month+1)]
    else:
        days_in_month = [get_days_in_month(day_start_year, month) for month in range(day_start_month, 13)]
        day_start_year = day_start_year+1
        while day_start_year < day_end_year:
            for month in range(1, 13):
                days_in_month.append(get_days_in_month(day_start_year, month))
                day_start_year = day_start_year+1
        for month in range(1, day_end_month+1):
                days_in_month.append(get_days_in_month(day_start_year, month))
    return days_in_month

def date_to_fraction(date, E_month):
    result = []
    days_in_month =  E_month
    #print(days_in_month)
    for d in date:
        year = d.year
        month = d.month
        day = d.day
        
        days_to_month = days_in_month[month - date.min().month]
        
        result.append(month + ((day-1)/days_to_month))
    return result

In [19]:
import pandas as pd
import numpy as np
def make_non_timeseries(diff_cycle, tank_seq):
    timelen = diff_cycle.index
    E_month =  day_in_month_set(timelen.min().year, timelen.min().month, timelen.max().year, timelen.max().month)
    H = nontime_date = date_to_fraction(timelen, E_month)
    W = diff_cycle.values
    date_to_fraction(timelen ,E_month)
    nontime_date = pd.DataFrame(nontime_date)
    F = np.concatenate([[H], [W]])
    
    tr_F = F.T
    H_H = H
    W_W = W
    H = np.array(H_H)
    W = np.array(W_W)

        # Sort H and W by ascending order of H
    sorted_indices = np.argsort(H)
    sorted_H = H[sorted_indices]
    sorted_W = W[sorted_indices]

        # Transpose the arrays
    F = np.vstack((sorted_H, sorted_W)).T
        #print(F)
    return F

In [24]:
import matplotlib.pyplot as plt
from numpy import arange, ones, pi
from scipy import cos, sin
from scipy.fftpack import fft, fftfreq, ifft
from scipy.optimize import curve_fit
import numpy as np
import pickle


def fsl(x,a,b,c,d,e,f):
    return a*x**5+b*x**4+c*x**3+d*x**2+e*x+f

def fourie_fitting(sdata, tank_seq):
    # 주파수 구하기
    num_coeff = 10
    N = sdata.shape[0]
    dt = sdata[1, 0] - sdata[0, 0]
    freq = np.fft.fftfreq(N, d=dt)

    # 푸리에 변환하기
    yf = fft(sdata[:, 1])
    xf = freq

    # 푸리에 근사치 구하기
    # 사용할 계수 수
    yf_approx = np.zeros_like
    yf_trunc = np.zeros_like(yf)
    yf_trunc[:num_coeff] = yf[:num_coeff]

        # 역변환
    y_approxd = np.real(ifft(yf_trunc))
        
    H = y_approxd
    W =  sdata[:, 1]
    popt, pcov = curve_fit(fsl, H, W)

    modelf = lambda x: fsl(x, *popt)
    popt_set = popt
    return popt_set, yf, num_coeff

In [36]:
import datetime
from datetime import datetime
import calendar
from scipy.fftpack import fft, fftfreq, ifft
def fslvalue_per_day(x0, x1, x2, tank_seq, yf, sdata):
    day_in_month = get_days_in_month(x0, x1)
    x = x1 + x2/(day_in_month+1)
    num_coeff = 10
    yfd_trunc = np.zeros_like(yf)
    yfd_trunc[:num_coeff] = yf[:num_coeff]
    y_interp = np.interp(x, sdata[:, 0], np.real(ifft(yfd_trunc)))
    return y_interp

def remainby_cycle(timestamp,tank_seq, remain, popt_set, yf, sdata):
    warning = [40, 30, 20, 10]
    pdict = {}
    remain = float(remain)
    timestamp = datetime.strptime(timestamp, "%Y-%m-%d")
    year = timestamp.year
    month = timestamp.month
    day = timestamp.day
    
    while remain >=10:
        fsl_cycle_value = fsl(fslvalue_per_day(year, month, day, tank_seq, yf, sdata), *popt_set)
        if fsl_cycle_value < min(sdata[:,1]):
            fsl_cycle_value = min(sdata[:,1])
        used_gas = 50*7/fsl_cycle_value
        if remain > warning[0] and remain - used_gas <= warning[0]:
            print(warning[0],"Warning", year,".", month,".",day, " : ",round(remain - used_gas,3))
            timestamp = datetime(year, month, day)
            adict = {'remain':round(remain,3),'timestamp':timestamp }
            pdict[str(warning[0])] = adict
        elif remain > warning[1] and remain - used_gas <= warning[1]:
            print(warning[1],"Warning", year,".", month,".",day, " : ",round(remain - used_gas,3))
            timestamp = datetime(year, month, day)
            adict = {'remain':round(remain,3),'timestamp':timestamp }
            pdict[str(warning[1])] = adict
        elif remain > warning[2] and remain - used_gas <= warning[2]:
            print(warning[2],"Warning", year,".", month,".",day, " : ",round(remain - used_gas,3))
            timestamp = datetime(year, month, day)
            adict = {'remain':round(remain,3),'timestamp':timestamp }
            pdict[str(warning[2])] = adict
        remain = remain - used_gas
        max_days = calendar.monthrange(year, month)[1]
        if remain <=warning [3]:
            break
        day += 1
        if day > max_days:
            month += 1
            day = 1
            if month > 12:
                year += 1
                month = 1
        
        
    print(year,".", month,".",day, " : ",round(remain,3))
    timestamp = datetime(year, month, day)
    adict = {'remain':round(remain,3),'timestamp':timestamp }
    pdict[str(warning[3])] = adict
    return pdict

In [37]:
def runmodel(x,  tank_seq,remain):
    datas = server_connection(tank_seq)
    datas_to_diff_cycle = datas_to_diff_cycle_data(datas, tank_seq)
    sdata = make_non_timeseries(datas_to_diff_cycle, tank_seq)
    fourie = fourie_fitting(sdata, tank_seq)
    popt_set = fourie[0]
    yf = fourie[1]
    
    return remainby_cycle(x,tank_seq, remain, popt_set, yf, sdata)

In [40]:
from datetime import datetime
from flask import Flask, jsonify
ds = '2022-08-05'
tank_seq = '333'
remain = 75

jup = runmodel(ds, tank_seq, remain)
print(jup)

40 Warning 2022 . 8 . 12  :  36.695
30 Warning 2022 . 8 . 14  :  25.314
20 Warning 2022 . 8 . 15  :  19.176
2022 . 8 . 17  :  5.893
{'40': {'remain': 41.919, 'timestamp': datetime.datetime(2022, 8, 12, 0, 0)}, '30': {'remain': 31.156, 'timestamp': datetime.datetime(2022, 8, 14, 0, 0)}, '20': {'remain': 25.314, 'timestamp': datetime.datetime(2022, 8, 15, 0, 0)}, '10': {'remain': 5.893, 'timestamp': datetime.datetime(2022, 8, 17, 0, 0)}}


In [43]:
from flask import Flask
from flask import request
from datetime import datetime
app = Flask(__name__)

@app.route('/test', methods=['POST'])
def seaqq():
    data = request.form
    remain = data['remain']
    tank_seq = data['tank_seq']
    timestamp = data['timestamp']

    runningvalue = runmodel(timestamp, tank_seq, remain)
    stringreturnvalue = str(runningvalue)
    print(stringreturnvalue)

    return jsonify({'result': runningvalue})

# @app.route('/join', methods=['POST'])
# def join_post():

#     name_receive = request.form['name_give']
#     print(name_receive)
#     return jsonify({'result':'success', 'msg': 'POST 요청!'})

if __name__ == '__main__':
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [15/May/2023 17:04:46] "POST /test HTTP/1.1" 400 -
127.0.0.1 - - [15/May/2023 17:05:04] "POST /test HTTP/1.1" 200 -


40 Warning 2022 . 8 . 20  :  37.578
30 Warning 2022 . 8 . 22  :  26.223
20 Warning 2022 . 8 . 25  :  17.927
2022 . 8 . 29  :  8.947
{'40': {'remain': 45.713, 'timestamp': datetime.datetime(2022, 8, 20, 0, 0)}, '30': {'remain': 30.668, 'timestamp': datetime.datetime(2022, 8, 22, 0, 0)}, '20': {'remain': 20.274, 'timestamp': datetime.datetime(2022, 8, 25, 0, 0)}, '10': {'remain': 8.947, 'timestamp': datetime.datetime(2022, 8, 29, 0, 0)}}
